In [1]:
!pip3 install -e ..
!pip3 install timm
!pip3 install wget
#install netron to visualize onnx model at localhost:8080, one has to run netron at command line
!pip3 install netron
#!pip3 install torch2trt
#!pip3 install nvidia-pyindex
#!pip3 install nvidia-tensorrt

Obtaining file:///w251_project
  Found existing installation: mids-plane-classification 0.0.1
    Can't uninstall 'mids-plane-classification'. No files were found to uninstall.
  Running setup.py develop for mids-plane-classification
    100% |################################| 1.5MB 292kB/s ta 0:00:01    14% |####                            | 215kB 4.6MB/s eta 0:00:01


In [2]:
#onnx conversion requirements
import os
import torch
import torchvision.models as models
from mids_plane_classification.models.resnet18 import AircraftResNet18
import onnx
#tensorrt and benchmark requirements
#import torch_tensorrt
import tensorrt as trt
import timm
import time
import numpy as np
import torch.backends.cudnn as cudnn
from mids_plane_classification.loaders.dataloader import PlaneDataModule

In [3]:
# set the device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('running on device ' + str(device))

running on device cuda:0


In [4]:
chk_path ='/w251_project/mids_plane_classification/models/checkpoints/model_best.pth.tar'
print('loading checkpoint:  ' + chk_path)
checkpoint = torch.load(chk_path)
arch = checkpoint['arch']

loading checkpoint:  /w251_project/mids_plane_classification/models/checkpoints/model_best.pth.tar


In [5]:
print('using model:  ' + arch)
model = AircraftResNet18()
model.load_state_dict(checkpoint['state_dict'])
epoch = checkpoint['epoch']

using model:  resnet18


In [6]:
#Should we try optimizer class if we are trying models in common mids library , would help this code to have that as class directly
#right now I have copied it from the training code, I dont know if we can directly import optimizer from checkpoint without declaring
#class object
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
optimizer.load_state_dict(checkpoint['optimizer'])

In [7]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight 	 torch.Size([64])
layer1.1.bn1.bias 	 torch.Size([64])
layer1.1.bn1.running_mean 	 torch.Size([64])
layer1.1.bn1.running_var 	 torch.Size([64])
layer1.1.bn1.num_batc

In [8]:
model.to(device)
model.eval()

AircraftResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [9]:
resolution = 256
input_img = torch.ones((1, 3, resolution, resolution)).cuda()
print('input size:  {:d}x{:d}'.format(resolution, resolution))

input size:  256x256


In [10]:
onnx_path ='/w251_project/mids_plane_classification/models'+ arch + '.onnx'

In [11]:
input_names = ["input_0"]
output_names = ["output_0"]

In [12]:
print('exporting model to ONNX...')
torch.onnx.export(model, input_img, onnx_path, verbose=True, input_names=input_names, output_names=output_names)
print('model exported to:  {:s}'.format(onnx_path))

exporting model to ONNX...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


graph(%input_0 : Float(1, 3, 256, 256, strides=[196608, 65536, 256, 1], requires_grad=0, device=cuda:0),
      %fc.weight : Float(11, 512, strides=[512, 1], requires_grad=1, device=cuda:0),
      %fc.bias : Float(11, strides=[1], requires_grad=1, device=cuda:0),
      %193 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %194 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %196 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %197 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %199 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %200 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %202 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %203 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %205 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %206 

In [13]:
import onnx
onnx_model = onnx.load('/w251_project/mids_plane_classification/modelsresnet18.onnx')
# Print a human readable representation of the graph
print('Network Graph:')
print(onnx.helper.printable_graph(onnx_model.graph))
print('')

# Print model metadata
print('ONNX version:      ' + onnx.__version__)
print('IR version:        {:d}'.format(onnx_model.ir_version))
print('Producer name:     ' + onnx_model.producer_name)
print('Producer version:  ' + onnx_model.producer_version)
print('Model version:     {:d}'.format(onnx_model.model_version))
print('')

# Check that the IR is well formed
print('Checking model IR...')
check = onnx.checker.check_model(onnx_model)
if check == None:
    print('The model was checked successfully!')

Network Graph:
graph torch-jit-export (
  %input_0[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 11x512]
  %fc.bias[FLOAT, 11]
  %193[FLOAT, 64x3x7x7]
  %194[FLOAT, 64]
  %196[FLOAT, 64x64x3x3]
  %197[FLOAT, 64]
  %199[FLOAT, 64x64x3x3]
  %200[FLOAT, 64]
  %202[FLOAT, 64x64x3x3]
  %203[FLOAT, 64]
  %205[FLOAT, 64x64x3x3]
  %206[FLOAT, 64]
  %208[FLOAT, 128x64x3x3]
  %209[FLOAT, 128]
  %211[FLOAT, 128x128x3x3]
  %212[FLOAT, 128]
  %214[FLOAT, 128x64x1x1]
  %215[FLOAT, 128]
  %217[FLOAT, 128x128x3x3]
  %218[FLOAT, 128]
  %220[FLOAT, 128x128x3x3]
  %221[FLOAT, 128]
  %223[FLOAT, 256x128x3x3]
  %224[FLOAT, 256]
  %226[FLOAT, 256x256x3x3]
  %227[FLOAT, 256]
  %229[FLOAT, 256x128x1x1]
  %230[FLOAT, 256]
  %232[FLOAT, 256x256x3x3]
  %233[FLOAT, 256]
  %235[FLOAT, 256x256x3x3]
  %236[FLOAT, 256]
  %238[FLOAT, 512x256x3x3]
  %239[FLOAT, 512]
  %241[FLOAT, 512x512x3x3]
  %242[FLOAT, 512]
  %244[FLOAT, 512x256x1x1]
  %245[FLOAT, 512]
  %247[FLOAT, 512x512x3x3]
  %248[FLOAT, 512]
  %250